In [133]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import pickle


In [134]:
## Load data
data = pd.read_csv('Churn_Modelling.csv')
data.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4215,4216,15574213,Wilson,789,France,Female,53,1,158271.74,1,1,1,5036.39,1
9749,9750,15767594,Azubuike,533,France,Female,35,8,0.00,2,1,1,187900.12,0
5484,5485,15595842,Paramor,748,Germany,Male,45,2,119852.01,1,0,0,73853.94,1
4546,4547,15738248,Lo,662,France,Female,37,5,0.00,2,1,0,151871.84,0
6396,6397,15602812,Holmes,684,Germany,Female,44,2,133776.86,2,0,1,49865.04,0


In [135]:
## preprocessing data
### Drop irrelevant columns  

'''note : Don't use inplace=True. Always assign back  data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)'''

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [136]:
### encode categorical variables

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.sample(4)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9011,775,France,1,42,6,133970.22,2,0,1,187839.90,0
3631,739,Spain,1,19,5,89750.21,1,1,0,193008.52,0
1097,543,France,1,36,6,0.00,2,1,0,176728.28,0
3612,635,Spain,0,37,3,0.00,2,1,0,91086.73,0


In [137]:
## one hot encode 'Geography' column

onehot_encode_geography = OneHotEncoder()
geography_encoder = onehot_encode_geography.fit_transform(data[['Geography']]).toarray()
geography_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [ ]:
onehot_encode_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [139]:
geography_encoder_df = pd.DataFrame(geography_encoder, columns=onehot_encode_geography.get_feature_names_out(['Geography']), index=data.index)

In [140]:
geography_encoder_df.sample(4)

,Geography_France,Geography_Germany,Geography_Spain
2939,1.0,0.0,0.0
2774,1.0,0.0,0.0
4011,0.0,1.0,0.0
2668,1.0,0.0,0.0


In [141]:
data = pd.concat([data.drop('Geography', axis=1), geography_encoder_df], axis=1)

In [142]:
data.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [143]:
## Save the encoders and Scalar  in pickel file

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encode_geography.pkl', 'wb') as file:
    pickle.dump(onehot_encode_geography, file)


In [144]:
## spliting dataset into indepent and dependent features 
X = data.drop('Exited', axis=1)
y = data['Exited']

##  Split the data in training and tetsing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test =  scaler.transform(X_test)

In [145]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [146]:
## save Scaler in pickel file
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [147]:
X_train[:3]

array([[ 0.35649971,  0.91324755, -0.6557859 ,  0.34567966, -1.21847056,
         0.80843615,  0.64920267,  0.97481699,  1.36766974,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, -0.3483691 ,  0.69683765,
         0.80843615,  0.64920267,  0.97481699,  1.6612541 , -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, -0.69539349,  0.61862909,
        -0.91668767,  0.64920267, -1.02583358, -0.25280688, -0.99850112,
        -0.57946723,  1.73494238]])

## ANN Implementaion

In [148]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [149]:
X_train.shape

(8000, 12)

In [150]:
## building model
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)),  ## Hidden layer connected with input layer
    Dense(32, activation = 'relu'),   ##Hidden layer 2
    Dense(1, activation = 'sigmoid')  ##output layer
]
)

In [151]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [152]:
# import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss =  tensorflow.keras.losses.BinaryCrossentropy()
loss

In [153]:
## compile model
model.compile(optimizer=opt, loss = 'binary_crossentropy', metrics=['accuracy'])

In [154]:
## set up Tensorboard
# from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [155]:
## set up Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [156]:
## Model Traning
history = model.fit(
    X_train, y_train, 
    validation_data = (X_test,y_test),
    epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.4035 - accuracy: 0.8298 - val_loss: 0.3516 - val_accuracy: 0.8610
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3561 - accuracy: 0.8540 - val_loss: 0.3442 - val_accuracy: 0.8635
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3509 - accuracy: 0.8555 - val_loss: 0.3425 - val_accuracy: 0.8590
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3428 - accuracy: 0.8565 - val_loss: 0.3407 - val_accuracy: 0.8605
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3386 - accuracy: 0.8606 - val_loss: 0.3355 - val_accuracy: 0.8645
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3344 - accuracy: 0.8627 - val_loss: 0.3580 - val_accuracy: 0.8575
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3381 - accuracy: 0.8595 - val_loss: 0.3522 - val_accuracy: 0.8585

In [157]:
model.save('model.h5')

c:\Users\aksha\AIML\supervised\Udemy\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [158]:
## load Tensorboard Extenstion
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [159]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 5364), started 0:56:47 ago. (Use '!kill 5364' to kill it.)